# Development Notebook

In [56]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/calendar']

def  get_calendar_service():
    flow = InstalledAppFlow.from_client_secrets_file(r"C:\Users\Issei\Documents\API Tokens\CalenderSecretary\client_secret_630503106817-im66k9rhjgpu3us9b8dvkhsi4j9qku7d.apps.googleusercontent.com.json", 
    SCOPES)
    creds = flow.run_local_server(port=8080)
    return build('calendar', 'v3', credentials=creds)

service = get_calendar_service()
print('Google Calendar APIに接続されました。')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=630503106817-im66k9rhjgpu3us9b8dvkhsi4j9qku7d.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=Bdh7HOzMQQPHeBNRcWieKyKYKg3e5g&access_type=offline
Google Calendar APIに接続されました。


In [62]:
from datetime import datetime, timedelta

def list_events(service):
    now = datetime.utcnow().isoformat() + 'Z'
    week_later = (datetime.utcnow() + timedelta(days=7)).isoformat() + 'Z'
    events_result = service.events().list(calendarId='primary', timeMin=now, timeMax=week_later, maxResults=5, singleEvents=True, orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print('直近の予定はありません。')
    else:
        print('予定：')
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            print(f"- {start}: {event['summary']}")

    return events

list_events(service)

予定：
- 2024-12-09T10:00:00+09:00: テストイベント
- 2024-12-10T09:00:00+09:00: テストイベント
- 2024-12-10T13:00:00+09:00: gptのテストイベント


[{'kind': 'calendar#event',
  'etag': '"3467306675308000"',
  'id': 'lcf69rrcfj26mojnv7ebmp54o8',
  'status': 'confirmed',
  'htmlLink': 'https://www.google.com/calendar/event?eid=bGNmNjlycmNmajI2bW9qbnY3ZWJtcDU0bzggbWF0Y2gubWF0c3V6b2VAbQ',
  'created': '2024-12-08T10:22:17.000Z',
  'updated': '2024-12-08T10:22:17.654Z',
  'summary': 'テストイベント',
  'creator': {'email': 'match.matsuzoe@gmail.com', 'self': True},
  'organizer': {'email': 'match.matsuzoe@gmail.com', 'self': True},
  'start': {'dateTime': '2024-12-09T10:00:00+09:00', 'timeZone': 'Asia/Tokyo'},
  'end': {'dateTime': '2024-12-09T11:00:00+09:00', 'timeZone': 'Asia/Tokyo'},
  'iCalUID': 'lcf69rrcfj26mojnv7ebmp54o8@google.com',
  'sequence': 0,
  'reminders': {'useDefault': True},
  'eventType': 'default'},
 {'kind': 'calendar#event',
  'etag': '"3467317500130000"',
  'id': 'uoie14tisasvn82tbhnlsv64ms',
  'status': 'confirmed',
  'htmlLink': 'https://www.google.com/calendar/event?eid=dW9pZTE0dGlzYXN2bjgydGJobmxzdjY0bXMgbWF0Y2gubW

In [27]:
def add_event(service, title, start_time, end_time=None):
    event = {
        'summary':title,
        'start': {'dateTime': start_time, 'timeZone': 'Asia/Tokyo'},
        'end': {'dateTime': end_time, 'timeZone': 'Asia/Tokyo'}
    }
    event_result = service.events().insert(calendarId='primary', body=event).execute()
    print(f'予定を追加しました。: {event_result.get("htmlLink")}, {event_result.get("id")}')

add_event(service, 'テストイベント', '2024-12-09T10:00:00', '2024-12-09T11:00:00')

予定を追加しました。: https://www.google.com/calendar/event?eid=bGNmNjlycmNmajI2bW9qbnY3ZWJtcDU0bzggbWF0Y2gubWF0c3V6b2VAbQ, lcf69rrcfj26mojnv7ebmp54o8


In [23]:
def update_event(service, event_id, new_title):
    event = service.events().get(calendarId='primary', eventId=event_id).execute()
    event['summary'] = new_title
    updated_event = service.events().update(calendarId='primary', eventId=event_id, body=event).execute()
    print(f"更新されたイベント: {updated_event['summary']}")

update_event(service, "6fv3os3p2br3n0mgsf96vq5cig", "テストイベント２")

更新されたイベント: テストイベント２


In [25]:
def list_free_times(service, period=3):
    now = datetime.utcnow().isoformat() + 'Z'
    xdays_later = (datetime.utcnow() + timedelta(days=period)).isoformat() + 'Z'
    events_result = service.events().list(calendarId='primary', timeMin=now, timeMax=xdays_later, maxResults=10, singleEvents=True, orderBy='startTime').execute()
    events = events_result.get('items', [])
    busy_times = []
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        end = event['end'].get('dateTime', event['end'].get('date'))
        busy_times.append({'start': start, 'end': end})
    print('予定されている時間：')
    for bt in busy_times:
        print(f"- {bt['start']} - {bt['end']}")
    free_times = []
    for i in range(len(busy_times) - 1):
        start = busy_times[i]['end']
        end = busy_times[i + 1]['start']
        free_times.append({'start': start, 'end': end})
    print('空いている時間：')
    for ft in free_times:
        print(f"- {ft['start']} - {ft['end']}")
    return free_times

list_free_times(service, 2)

予定されている時間：
空いている時間：


[]

In [58]:
from openai import OpenAI

with open(r"C:\Users\Issei\Documents\API Tokens\ChatGPT.txt", 'r') as file:
    OPENAI_API_KEY = file.read().rstrip()

client = OpenAI(api_key=OPENAI_API_KEY)

def suumarize_events(events):
    event_text = "\n".join([
        f"{event['start'].get('dateTime', event['start'].get('date'))}: {event['summary']}" for event in events
    ])

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a professional secretary. Summarize the following events in japanese."},
            {"role": "user", "content": f"次のスケジュールを要約して:\n{event_text}"}
        ]
    )
    return response.choices[0].message.content

summary = suumarize_events(list_events(service))
print(summary)

予定：
- 2024-12-09T10:00:00+09:00: テストイベント
2024年12月9日午前10時: テストイベントがあります。


In [61]:
import json
def parse_and_add_event(user_input, service=service):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """
                You are a professional secretary. Parse the following text and add the event to the calendar.
                Output Format: {"title": Event Name", "start": "YYYY-MM-DDTHH:MM:SS", "end": "YYYY-MM-DD-THH:MM:SS}
                If the end time is not specified, it will be set to 1 hour after the start time.
                """
            },
            {"role": "user", "content": f"today is {datetime.utcnow().isoformat() + 'Z'}: {user_input}"}
        ]
    )
    parsed_text = response.choices[0].message.content
    print(f"パースされたテキスト: {parsed_text}")
    try:
        parsed_text = json.loads(parsed_text)
        print(f"辞書にパースされたテキスト: {parsed_text}")
        add_event(service, parsed_text['title'], parsed_text['start'], parsed_text['end'])
    except json.JSONDecodeError as e:
        print(f"エラー: {e}")
        return


parse_and_add_event("予定は明後日の13時から16時のgptのテストイベントです。", service)

パースされたテキスト: {"title": "gptのテストイベント", "start": "2024-12-10T13:00:00", "end": "2024-12-10T16:00:00"}
辞書にパースされたテキスト: {'title': 'gptのテストイベント', 'start': '2024-12-10T13:00:00', 'end': '2024-12-10T16:00:00'}
予定を追加しました。: https://www.google.com/calendar/event?eid=NG1tcXRoZ3RqdDZtNmFpZXAzNGtoaDkzYWcgbWF0Y2gubWF0c3V6b2VAbQ, 4mmqthgtjt6m6aiep34khh93ag
